<a href="https://colab.research.google.com/github/DDDS18-GTFS/ddds.18.capstone/blob/main/GTFS_ABQ_load_pickle_set_params_merge_fix_set_anomalies_v9_1ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ABC note 250726-1715: I have some concerns about the anomaly placement, but I also haven't worked with a multi-day dataset previously, so I need to do some more testing.


#Load the Libraries

In [ ]:
#Required Libraries
import pandas as pd
import numpy as np

# # Install if needed
# !pip install -q ipywidgets

import ipywidgets as widgets
from IPython.display import display, clear_output

from datetime import datetime
import zipfile

from shapely.geometry import Point, LineString
from geopy.distance import geodesic

from folium import Map, FeatureGroup, CircleMarker, PolyLine, Marker, Icon, LayerControl
from matplotlib import colors as mcolors
import matplotlib.pyplot as plt
import folium

!pip install psycopg2-binary pandas sqlalchemy

import psycopg2
from sqlalchemy import create_engine
import os
from google.colab import userdata

In [ ]:
# Database connection parameters
DB_NAME = "abq-transit-db"

# You'll need to store your database password/paramenters in Colab Secrets
DB_HOST = userdata.get('DB_HOST_2')
DB_PORT = userdata.get('DB_PORT')
DB_PASSWORD = userdata.get('DB_PASSWORD')
DB_USER = userdata.get("DB_USER")


# Create connection string
connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create SQLAlchemy engine
engine = create_engine(connection_string)

In [ ]:
# Select Parameters for Anomaly Detection
# This section sets threshholds for classifying anomalous transit events.

# --- Use meters for user input ---
# The jump_thresh_m_widget creates a sliding bar for selecting how much distance there must be between
# two consecutive positions for a single vehicle to have gone 'unusually far' or 'jumped'.
jump_thresh_m_widget = widgets.IntSlider(
    value=500,
    min=50,
    max=2000,
    step=50,
    description='Jump Distance',
    layout=widgets.Layout(width='60%')
)

# Other anomaly widgets
# The disappear_thresh_widget creates a sliding bar for selecting how many consecutive seconds  must pass
# without a vehicle transmitting data before we consider that vehicle to have been gone 'unusually long'.
disappear_thresh_widget = widgets.IntSlider(value=300, min=60, max=1800, step=30, description='Time Gap')

# The min_jump_widget creates a sliding bar for selecting how many 'jumps' a vehicle must make before
# we consider the jumping to be 'unusual'.
# SUGGESTION: can we make this based off of jump points as a percentage of total recorded points a required number of consecutive jumps?
# As we use more data, any set value of total jumps will become more and more easily met.
min_jump_widget = widgets.IntSlider(value=2, min=1, max=10, step=1, description='Min Jumps')

# The speed_thresh_widget creates a sliding bar for selecting how fast a vihicke must be going for us
# to consider that vehicle as 'unusually fast'.
speed_thresh_widget = widgets.IntSlider(value=70, min=10, max=100, step=5, description='Speed')

# motion to depricate this widget
reversal_thresh_widget = widgets.IntSlider(value=120, min=60, max=180, step=5, description='Heading Δ°')

# --- Stuck vehicle detection ---
# The stuck_speed_widget creates a sliding bar for selecting how slow a vehicle must be going
# for us to consider it as effectively unmoving.
stuck_speed_widget = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=5.0,
    step=0.1,
    description='Stuck Speed',
    layout=widgets.Layout(width='60%')
)

# The stuck_window_widget creates a sliding bar for selecting how many consecutive Stuck Speed
# events a vehicle must have before it is considered to have been still for an unusually long period.
stuck_window_widget = widgets.IntSlider(
    value=4,
    min=1,
    max=20,
    step=1,
    description='Window Size',
    layout=widgets.Layout(width='60%')
)

# --- Repeated point tolerance ---
# motion to depricate this widget
repeat_tolerance_widget = widgets.FloatLogSlider(
    value=1e-5,
    base=10,
    min=-7,  # 1e-7
    max=-3,  # 1e-3
    step=0.1,
    description='Repeat Tolerance',
    layout=widgets.Layout(width='60%')
)

# --- Early appearance margin ---
# motion to depricate this widget
early_margin_widget = widgets.IntSlider(
    value=30,
    min=0,
    max=600,
    step=10,
    description='Early Margin',
    layout=widgets.Layout(width='60%')
)

# --- Off-route buffer ---
# The offroute_buffer_widget creates a sliding bar for selecting how many meters
# a vehicle must be off of its expected route before we actually label it as off route
offroute_buffer_widget = widgets.IntSlider(
    value=50,
    min=10,
    max=500,
    step=10,
    description='Off-Route Buffer',
    layout=widgets.Layout(width='60%')
)

# Labels
jump_label = widgets.Label(value="(meters)")
disappear_label = widgets.Label(value="(seconds)")
min_jump_label = widgets.Label(value="(count)")
speed_label = widgets.Label(value="(mph)")
reversal_label = widgets.Label(value="(degrees of reversal)")
stuck_speed_label = widgets.Label(value="(mph)")
stuck_window_label = widgets.Label(value="(frames in rolling window)")
repeat_tolerance_label = widgets.Label(value="(decimal degrees)")
early_margin_label = widgets.Label(value="(seconds)")
offroute_buffer_label = widgets.Label(value="(meters)")

# Assemble UI layout
slider_widgets = widgets.VBox([
    widgets.HBox([jump_thresh_m_widget, jump_label]),
    widgets.HBox([disappear_thresh_widget, disappear_label]),
    widgets.HBox([min_jump_widget, min_jump_label]),
    widgets.HBox([speed_thresh_widget, speed_label]),
    widgets.HBox([reversal_thresh_widget, reversal_label]),
    widgets.HBox([stuck_speed_widget, stuck_speed_label]),
    widgets.HBox([stuck_window_widget, stuck_window_label]),
    widgets.HBox([repeat_tolerance_widget, repeat_tolerance_label]),
    widgets.HBox([early_margin_widget, early_margin_label]),
    widgets.HBox([offroute_buffer_widget, offroute_buffer_label]),
])

# Button and save logic
submit_button = widgets.Button(description="Save Parameters", button_style='primary')
anomaly_params = {}

def save_params(b):
    '''
    save_params displays widgets that allow you to set up and save anomaly detection threshholds.
    '''
    clear_output(wait=True)
    display(slider_widgets, submit_button)

    global anomaly_params
    jump_m = jump_thresh_m_widget.value
    jump_deg = jump_m / 111000  # Convert meters → degrees

    anomaly_params = {
        "JUMP_DISTANCE_THRESHOLD": jump_deg,
        "JUMP_DISTANCE_METERS": jump_m,
        "DISAPPEARANCE_TIME_THRESHOLD": disappear_thresh_widget.value,
        "MIN_JUMP_COUNT_PER_VEHICLE": min_jump_widget.value,
        "SPEED_LIMIT_MPH": speed_thresh_widget.value,
        "REVERSAL_HEADING_THRESHOLD": reversal_thresh_widget.value,
        "STUCK_SPEED_MPH": stuck_speed_widget.value,
        "STUCK_WINDOW_SIZE": stuck_window_widget.value,
        "REPEATED_COORD_TOLERANCE": repeat_tolerance_widget.value,
        "EARLY_APPEARANCE_MARGIN_SEC": early_margin_widget.value,
        "OFF_ROUTE_BUFFER_M": offroute_buffer_widget.value,
    }

    print("✅ Anomaly detection parameters set:")
    for k, v in anomaly_params.items():
        print(f"  {k}: {v}")

submit_button.on_click(save_params)

# Display interface
display(slider_widgets, submit_button)


#LOAD PICKLE

In [ ]:
# This cell should be used when loading pickle files as opposed to using the database
#import pickle

# Load a pickle file of a dataframe which contains our historic transit data.
#with open("captures.p", "rb") as f:
#    captured_data = pickle.load(f)


In [ ]:
with engine.connect() as connection:
    captured_data = pd.read_sql_query(
        '''
        SELECT
          id, snapshot_id, msg_time, timestamp_collected, vehicle_id, location, heading, speed_mph,
          route_short_name, trip_id, next_stop_id, next_stop_id, next_stop_name,
          next_stop_sched_time
        FROM
          vehicle_snapshots
        WHERE
          DATE("timestamp_collected") = CURRENT_DATE - INTERVAL '1 DAY'
        ORDER BY
          timestamp_collected DESC;
        ''', connection)

captured_data.head()

##Data Validation

In [ ]:
cd_df = captured_data
cd_df.size

In [ ]:
# cd_df.head(20)

In [ ]:
# # Display full list of column names
# print("Number of columns:", len(cd_df.columns))
# print("Column names:\n", cd_df.columns.tolist())

In [ ]:
# cd_df.head()

In [ ]:
# Convert timestamp to datetime
cd_df['timestamp_collected'] = pd.to_datetime(cd_df['timestamp_collected'], errors='coerce')

# 1. Shape and Column Overview
print("Shape:", cd_df.shape)
print("Columns:", cd_df.columns.tolist())

# 2. Null/Missing Value Counts
print("\nMissing Values:\n", cd_df.isnull().sum())

# 3. Duplicate Detection
duplicates = cd_df.duplicated().sum()
print("\nDuplicate rows:", duplicates)

# 4. Timestamp Range
print("\nTimestamp Range:")
print("Min:", cd_df['timestamp_collected'].min())
print("Max:", cd_df['timestamp_collected'].max())
print("Duration:", cd_df['timestamp_collected'].max() - cd_df['timestamp_collected'].min())

# 5. Latitude/Longitude Range
print("\nLatitude Range:", cd_df['latitude'].min(), "to", cd_df['latitude'].max())
print("Longitude Range:", cd_df['longitude'].min(), "to", cd_df['longitude'].max())

# 6. Speed Summary
print("\nSpeed Summary:\n", cd_df['speed_mph'].describe())


In [ ]:
# # Plot the histogram and get the counts and bin edges
# counts, bins, patches = plt.hist(cd_df['longitude'], bins=2, rwidth=0.8)

# # Add labels and title
# plt.xlabel("Value")
# plt.ylabel("Count")
# plt.title("Histogram of Values with Counts per Bin")

# # Display the plot
# plt.show()

# # You can also access the counts and bin edges directly:
# print("Bin counts:", counts)
# print("Bin edges:", bins)

In [ ]:
# # Plot the histogram and get the counts and bin edges
# counts, bins, patches = plt.hist(cd_df['latitude'], bins=2, rwidth=0.8)

# # Add labels and title
# plt.xlabel("Value")
# plt.ylabel("Count")
# plt.title("Histogram of Values with Counts per Bin")

# # Display the plot
# plt.show()

# # You can also access the counts and bin edges directly:
# print("Bin counts:", counts)
# print("Bin edges:", bins)

In [ ]:
# Select rows where 'longitude' and 'latitude' have the same value
rows_with_same_values = cd_df[cd_df['longitude'] == cd_df['latitude']]
rows_without_same_values = cd_df[cd_df['longitude'] != cd_df['latitude']]

# Print the resulting DataFrame
print(rows_with_same_values)

In [ ]:
# rows_without_same_values.shape

In [ ]:
# cd_df.shape

####*.v8.2 update
We're removing the off-duty rows where Lat = Long = 0

In [ ]:
cd_df_copy = cd_df.copy()
cd_df_copy.shape

In [ ]:
cd_df = cd_df[cd_df['longitude'] != cd_df['latitude']]
cd_df.shape

##✅ Overall Status
Your dataset appears valid and cleanly structured for analysis. You can safely proceed with anomaly detection, with only minor issues to address or monitor.

| Category            | Result                               | Notes                                                                          |
| ------------------- | ------------------------------------ | ------------------------------------------------------------------------------ |
| **Shape**           | (1,096,540 rows, 12 columns)         | Substantial volume, well-sized for analysis                                    |
| **Missing Values**  | 0 missing in any column              | ✅ Excellent data integrity                                                     |
| **Duplicates**      | 0 duplicate rows                     | ✅ Clean snapshot logic                                                         |
| **Timestamp Range** | \~1.6 days                           | ⚠️ Slightly under 2 days; may be due to gaps or partial logging                |
| **Latitude Range**  | `0.0` to `35.41`                     | ⚠️ `0.0` indicates corrupted or failed GPS points — needs filtering            |
| **Longitude Range** | `-106.79` to `0.0`                   | ⚠️ Same issue — longitudes of `0.0` are invalid (off the west coast of Africa) |
| **Speed (mph)**     | Mean: `4.2`, Max: `80`, 75% at `0.0` | ⚠️ Excessive zero speeds → vehicles idle or unreported? Worth mapping          |
| **Outliers**        | Max speed = `80 mph`                 | 🔍 Borderline — worth flagging for manual inspection                           |


In [ ]:
# #Some preliminary prep for analysis

# # Remove invalid GPS coordinates
# cd_df_valid = cd_df[(cd_df['latitude'] > 30) & (cd_df['longitude'] < -100)]

# # Optional: filter out idle snapshots (speed = 0) if focusing on movement
# cd_df_moving = cd_df_valid[cd_df_valid['speed_mph'] > 0]


#Convert code for snapshot to use Pickle

In [ ]:
def filter_cd_df(df, start_time=None, end_time=None, routes=None, vehicles=None, bounds=None):
    '''
    Filters a dataframe(df) such that:
    - the "timestamp_collected" column's entries are greater than or equal to the start_time
    - the "timestamp_collected" column's entries are less than or equal to the end_time
    - the "route_short_name" column's entries are in routes
    - the "vehicle_id" column's entries are in vehicles

    Returns the filtered dataframe.
    '''
    df_filtered = df.copy()

    if start_time:
        df_filtered = df_filtered[df_filtered['timestamp_collected'] >= pd.to_datetime(start_time)]
    if end_time:
        df_filtered = df_filtered[df_filtered['timestamp_collected'] <= pd.to_datetime(end_time)]
    if routes:
        df_filtered = df_filtered[df_filtered['route_short_name'].isin(routes)]
    if vehicles:
        df_filtered = df_filtered[df_filtered['vehicle_id'].isin(vehicles)]
    if bounds:
        lat_min, lat_max, lon_min, lon_max = bounds
        df_filtered = df_filtered[
            (df_filtered['latitude'] >= lat_min) & (df_filtered['latitude'] <= lat_max) &
            (df_filtered['longitude'] >= lon_min) & (df_filtered['longitude'] <= lon_max)
        ]

    return df_filtered


In [ ]:


# --- Widgets ---
# route_widget lets you select which routes you'd like to like to plot in folium from a menu.
route_widget = widgets.SelectMultiple(
    options=sorted(cd_df['route_short_name'].unique()),
    description='Routes',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)
# vehicle_widget lets you select which vehicles' data you'd like to plot in folium from a menu.
vehicle_widget = widgets.SelectMultiple(
    options=sorted(cd_df['vehicle_id'].unique()),
    description='Vehicles',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)
# start_widget lets you select the earliest date you'd like to use data from.
start_widget = widgets.DatePicker(
    description='Start Date',
    value=cd_df['timestamp_collected'].min().date()
)
# end_widget lets you select the latest date you'd like to use data from.
end_widget = widgets.DatePicker(
    description='End Date',
    value=cd_df['timestamp_collected'].max().date()
)

# hour_slider creates a set of sliding bars for selecting the earliest and latest
# point of the day that you'd like to use data from.
hour_slider = widgets.IntRangeSlider(
    value=[0, 23],
    min=0,
    max=23,
    step=1,
    description='Hour Range',
    layout=widgets.Layout(width='60%')
)

# Makes a button, that will later be used to apply filters.
filter_button = widgets.Button(description='Apply Filters', button_style='primary')

# --- Global to store result ---
cd_df_filtered = pd.DataFrame()

# --- Filtering callback ---
def apply_filters(b):
    '''
    Uses previously defined widgets to filter a dataframe of historical transit data
    by rout_short_name, vehicle_id, and timestamp collected.
    Prints the number of filtered rows and displays the first 10 rows of the filtered dataframe.
    '''
    global cd_df_filtered
    clear_output(wait=True)
    display(route_widget, vehicle_widget, start_widget, end_widget, hour_slider, filter_button)

    df = cd_df.copy()

    # Apply filters
    if route_widget.value:
        df = df[df['route_short_name'].isin(route_widget.value)]
    if vehicle_widget.value:
        df = df[df['vehicle_id'].isin(vehicle_widget.value)]

    start_dt = pd.to_datetime(start_widget.value)
    end_dt = pd.to_datetime(end_widget.value) + pd.Timedelta(days=1)
    df = df[(df['timestamp_collected'] >= start_dt) &
            (df['timestamp_collected'] < end_dt)]

    hr_start, hr_end = hour_slider.value
    df = df[(df['timestamp_collected'].dt.hour >= hr_start) &
            (df['timestamp_collected'].dt.hour <= hr_end)]

    # Store result globally
    cd_df_filtered = df

    print(f"✅ Filtered {len(df)} rows.")
    display(df.head(10))

# Displays a filter button that calls the 'apply_filters' function
filter_button.on_click(apply_filters)

# --- Display UI ---
display(route_widget, vehicle_widget, start_widget, end_widget, hour_slider, filter_button)


In [ ]:
# #Sanity check following filtering
# cd_df_filtered.describe()
# # or
# cd_df_filtered['vehicle_id'].value_counts()


In [ ]:
#Pickle replacement
df_new = cd_df_filtered

#Old code:
#✅ Step 1a: Load and Inspect the New Snapshot
# new_snapshot_path = "/content/cabq_gtfs_snapshots_20250722_1415.csv"
# df_new = pd.read_csv(new_snapshot_path)
# df_new.info()
# df_new.head(3)

# #Also print the columns:
# print(df_new.columns.tolist())


In [ ]:
#✅ Step 2a: Trip ID Validity
# Counts and displays the total rows and number of rows with invalid trip ids.
# Calculates and displays the percentage of rows with valid trip ids.
df_new["trip_id"] = df_new["trip_id"].astype(str)
invalid_trip_ids = df_new["trip_id"].isin(["0", "Undetermined", "nan", "", "None"]).sum()
total_rows = len(df_new)

print(f"Total rows: {total_rows}")
print(f"Invalid trip_ids: {invalid_trip_ids}")
print(f"Percent valid trip_ids: {100 * (total_rows - invalid_trip_ids) / total_rows:.2f}%")


In [ ]:
#*.v8.2 new calc for finding mostly Off Duty
invalid_route_ids = df_new["route_short_name"].isin(["0", "Undetermined", "nan", "", "None", "Off Duty"]).sum()
print(f"Invalid route_ids: {invalid_route_ids}")
print(f"Percent valid route_ids: {100 * (total_rows - invalid_route_ids) / total_rows:.2f}%")

In [ ]:
invalid_ids = ["0", "Undetermined", "nan", "", "None", "Off Duty"]
df_clean_test = df_new[~df_new["trip_id"].isin(invalid_ids) & ~df_new["route_short_name"].isin(invalid_ids)].copy()
df_clean_test.shape

In [ ]:
#✅ Step 2b: Filter invalid trip_ids
invalid_trip_ids = ["0", "Undetermined", "nan", "", "None"]
df_clean_trips = df_new[~df_new["trip_id"].isin(invalid_trip_ids)].copy()
df_clean_trips.shape

In [ ]:
#So after all that we can see that all of the rows dropped by df_clean_trips cover those dropped by df_clean_test (both invalid trips and routes)
invalid_route_ids = df_clean_trips["route_short_name"].isin(["0", "Undetermined", "nan", "", "None", "Off Duty"]).sum()
print(f"Invalid route_ids: {invalid_route_ids}")
print(f"Percent valid route_ids: {100 * (total_rows - invalid_route_ids) / total_rows:.2f}%")

In [ ]:
df_clean = df_clean_test

In [ ]:
# df_clean.head()

####Cleanup note
We can skip several of the above cells, as dropping the invalid trip_ids also drops the invalid route_short_names

#Load the Static data

In [ ]:
#✅ Step 1b: Reload Static GTFS and Normalize It (ensure data types are consistent)

# Adjust if needed — make sure this is the static feed aligned with 2025-07-22
gtfs_zip_path = "/content/google_transit.zip"

with zipfile.ZipFile(gtfs_zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/gtfs_static")

trips = pd.read_csv("/content/gtfs_static/trips.txt", dtype=str)
routes = pd.read_csv("/content/gtfs_static/routes.txt", dtype=str)


In [ ]:
# trips.shape

In [ ]:
# trips.head()

In [ ]:
# trips.describe()

In [ ]:
# routes.shape

In [ ]:
# routes.head()

In [ ]:
# routes.describe(include='all')

In [ ]:
#✅ Step 3b: Merge with trips.txt to Get route_id
# Take all of df_clean and add on data from trips if the trip_id matches across both dataframes.
df_with_trips = df_clean.merge(trips, on="trip_id", how="left")


In [ ]:
# df_with_trips.head()

In [ ]:
# df_with_trips.columns

####Merge check
1. Trip-Based Merges (Correct in Isolation)
- df_with_trips = df_clean.merge(trips, on="trip_id", how="left")
- df_full = df_with_trips.merge(routes, on="route_id", how="left")

✅ Good practice: merges are on appropriate keys (trip_id, route_id), and types are cast explicitly.

In [ ]:
# trips.columns

In [ ]:
# routes.columns

In [ ]:
# routes.head()

In [ ]:
#This cell caused some issues in the merge because the df_with_trips and routes dfs did not agree on route_short_name, so we are going to take another approach
# #✅ Step 4b: Merge with routes.txt to Get Descriptive Info
# df_with_trips["route_id"] = df_with_trips["route_id"].astype(str)

# # Ensure consistent types
# trips["shape_id"] = trips["shape_id"].astype(str)
# routes["route_id"] = routes["route_id"].astype(str)

# # Take all of df_with_trips and add on data from routes if the route_id matches across both dataframes.
# df_full = df_with_trips.merge(routes, on="route_id", how="left")


In [ ]:
#Let's avoid the route_..._x and route_..._y situation on the merge:
#✅ Step 4b: Merge with routes.txt to Get Descriptive Info
df_with_trips["route_id"] = df_with_trips["route_id"].astype(str)

# Ensure consistent types
trips["shape_id"] = trips["shape_id"].astype(str)
routes["route_id"] = routes["route_id"].astype(str)

# Assume 'df_with_trips' and 'routes' both contain 'route_id' and 'route_short_name'

# Step 1: Check 1:1 match on overlapping column
merged_check = df_with_trips[['route_id', 'route_short_name']].merge(
    routes[['route_id', 'route_short_name']],
    on='route_id',
    how='inner',
    suffixes=('_df_with_trips', '_routes')
)


In [ ]:
# merged_check.head()

In [ ]:
# merged_check.info()

In [ ]:
# Step 2: Find any mismatches
mismatch = merged_check[merged_check['route_short_name_df_with_trips'] != merged_check['route_short_name_routes']]

if not mismatch.empty:
    raise ValueError(f"Mismatch found in route_short_name across dataframes:\n{mismatch}")

In [ ]:
#Okay... so there are some mismatches between the data and the static routes info; let's see what they are
mismatch_summary = (
    mismatch[['route_id', 'route_short_name_df_with_trips', 'route_short_name_routes']]
    .drop_duplicates()
    .sort_values('route_id')
)
display(mismatch_summary)


#This is the stopping point for 250731-1200; pick up here

In [ ]:
# [k for k, v in globals().items() if str(type(v)) == "<class 'pandas.core.frame.DataFrame'>"]


In [ ]:
# routes.head()


In [ ]:
# routes.columns

In [ ]:
# Rename to preserve both versions of route_short_name
routes_renamed = routes.rename(columns={'route_short_name': 'route_short_name_static'})

# Merge with df_with_trips, keeping the RT version as primary
merged = df_with_trips.merge(routes_renamed, on='route_id', how='left')


In [ ]:
# merged.columns

In [ ]:
df_full = merged

In [ ]:
# df_full.describe(include='all')

In [ ]:
# #✅ Step 5b: Load shapes.txt
# shapes = pd.read_csv("/content/gtfs_static/shapes.txt", dtype={"shape_id": str})

In [ ]:
# shapes.describe(include='all')

In [ ]:
# shapes.head(1000)

In [ ]:
#✅ Step 5b: Load shapes.txt
shapes = pd.read_csv("/content/gtfs_static/shapes.txt", dtype={"shape_id": str})

# Build LineStrings for each shape_id
shape_lines = {}
for shape_id, group in shapes.groupby("shape_id"):
    sorted_group = group.sort_values("shape_pt_sequence")
    coords = list(zip(sorted_group["shape_pt_lon"], sorted_group["shape_pt_lat"]))
    shape_lines[shape_id] = LineString(coords)

# Merge trips and routes to link shape_id to route_short_name
shape_route_map = (
    trips.merge(routes, on="route_id", how="left")
         .dropna(subset=["route_short_name"])
         .drop_duplicates(subset=["shape_id"])
         .set_index("shape_id")[["route_id", "route_short_name"]]
)

In [ ]:
#✅ Step 6b: Create a data_quality Flag
def classify_row(row):
    if row["trip_id"] in invalid_trip_ids:
        return "invalid_trip_id"
    elif pd.isna(row["route_id"]):
        return "missing_route_id"
    elif pd.isna(row["route_long_name"]):
        return "missing_route_metadata"
    else:
        return "valid"

df_full["data_quality"] = df_full.apply(classify_row, axis=1)
print(df_full["data_quality"].value_counts())


#####Possible Deprication

In [ ]:
#Not sure if this is needed
# #Load the trips, routes, shapes from Static data
# with zipfile.ZipFile(gtfs_zip_path, 'r') as z:
#     trips_df = pd.read_csv(z.open("trips.txt"))
#     routes_df = pd.read_csv(z.open("routes.txt"))
#     # Load GTFS shapes.txt into a DataFrame
#     shapes_df = pd.read_csv(z.open("shapes.txt"))

#Clean the RT data

In [ ]:
#🔹 1.1 Filter for Valid Rows
df_valid = df_full[df_full["data_quality"] == "valid"].copy()

#🔹 1.2 Parse Timestamps
df_valid["timestamp"] = pd.to_datetime(df_valid["timestamp_collected"], utc=True)

#🔹 1.3 Sort by Vehicle and Timestamp
df_valid = df_valid.sort_values(by=["vehicle_id", "timestamp"])

#🔹 1.4 Organize by Vehicle
#This creates a dictionary keyed by vehicle ID, each with a sorted DataFrame:
vehicle_groups = dict(tuple(df_valid.groupby("vehicle_id")))

#You can confirm how many distinct vehicles you’re tracking:
print("Vehicle count:", len(vehicle_groups))


In [ ]:
# df_full.shape

In [ ]:
# df_full.describe(include='all')

In [ ]:
# df_full.head()

In [ ]:
# df_valid.shape

In [ ]:
# df_valid.describe(include='all')

In [ ]:
# df_valid.head(1000)

#Anomly Detection

🔹 1. Detect Jumps and Gaps

Already implemented, but here’s the modular form:

In [ ]:
# # 🔹 1. Detect Jumps and Gaps
# def detect_jumps_and_gaps(df, params):
#     """
#     Detects jump or gap anomalies based on distance (meters) and time (seconds)
#     thresholds pulled from the anomaly_params dictionary.

#     Parameters:
#         df (pd.DataFrame): Filtered DataFrame for a single vehicle
#         params (dict): Anomaly detection thresholds from widget interface

#     Returns:
#         List[dict]: List of detected jump or gap anomalies
#     """
#     distance_threshold = params["JUMP_DISTANCE_METERS"]           # in meters
#     time_threshold = params["DISAPPEARANCE_TIME_THRESHOLD"]       # in seconds

#     anomalies = []
#     for i in range(1, len(df)):
#         row_prev, row_curr = df.iloc[i - 1], df.iloc[i]
#         time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()

#         distance = geodesic(
#             (row_prev["latitude"], row_prev["longitude"]),
#             (row_curr["latitude"], row_curr["longitude"])
#         ).meters

#         if time_diff > time_threshold or distance > distance_threshold:
#             anomalies.append({
#                 "vehicle_id": row_curr["vehicle_id"],
#                 "timestamp_prev": row_prev["timestamp"],
#                 "timestamp_curr": row_curr["timestamp"],
#                 "time_diff_sec": time_diff,
#                 "distance_m": distance,
#                 "is_gap": time_diff > time_threshold,
#                 "is_jump": distance > distance_threshold,
#                 "anomaly_type": "jump_or_gap"
#             })

#     return anomalies
# #Note: changed row["timestamp_collected"] to row["timestamp"] throughout, assuming that we've already converted timestamp_collected to UTC and assigned it to a new "timestamp" column at the start of the anomaly pipeline (which the existing pipeline does). This avoids inconsistency.

In [ ]:
# 🔹 1. Detect Jumps and Gaps – WITHIN trip_id
def detect_jumps_and_gaps(df, params):
    """
     Detects jump or gap anomalies based on distance (meters) and time (seconds)
     thresholds pulled from the anomaly_params dictionary.

    Parameters:
        df (pd.DataFrame): Filtered DataFrame for a single vehicle, sorted by timestamp.
                           Must include 'trip_id', 'timestamp', 'latitude', 'longitude'.
        params (dict): Anomaly detection thresholds from widget interface.

    Returns:
        List[dict]: Detected jump or gap anomalies.
    """
    distance_threshold = params["JUMP_DISTANCE_METERS"]           # meters
    time_threshold = params["DISAPPEARANCE_TIME_THRESHOLD"]       # seconds

    anomalies = []
    for i in range(1, len(df)):
        row_prev, row_curr = df.iloc[i - 1], df.iloc[i]

        # 💡 Trip boundary check — skip if different trip
        if row_prev["trip_id"] != row_curr["trip_id"]:
            continue

        time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()

        distance = geodesic(
            (row_prev["latitude"], row_prev["longitude"]),
            (row_curr["latitude"], row_curr["longitude"])
        ).meters

        if time_diff > time_threshold or distance > distance_threshold:
            anomalies.append({
                "vehicle_id": row_curr["vehicle_id"],
                "trip_id": row_curr["trip_id"],  # optional but useful for tracing
                "timestamp_prev": row_prev["timestamp"],
                "timestamp_curr": row_curr["timestamp"],
                "time_diff_sec": time_diff,
                "distance_m": distance,
                "is_gap": time_diff > time_threshold,
                "is_jump": distance > distance_threshold,
                "anomaly_type": "jump_or_gap"
            })

    return anomalies


In [ ]:
# #2. Detect Stuck Vehicles
# def detect_stuck_vehicle(df, params):
#     """
#     Detects stuck vehicles using a rolling window of speed and spatial consistency,
#     segmented by trip_id to avoid cross-trip contamination.

#     Parameters:
#         df (pd.DataFrame): Must contain 'speed_mph', 'latitude', 'longitude', and 'trip_id'.
#         params (dict): Widget-defined anomaly thresholds.

#     Returns:
#         pd.DataFrame: Rows flagged as 'stuck_vehicle'.
#     """
#     speed_thresh = params.get("STUCK_SPEED_MPH", 1.0)   # Default: 1 mph
#     window = params.get("STUCK_WINDOW_SIZE", 4)         # Default: 4-frame window

#     stuck_flags = (
#         (df["speed_mph"].rolling(window).mean() < speed_thresh) &
#         (df["latitude"].diff().abs().rolling(window).mean() < 0.0001) &
#         (df["longitude"].diff().abs().rolling(window).mean() < 0.0001)
#     )

#     return df[stuck_flags.fillna(False)].assign(anomaly_type="stuck_vehicle")


In [ ]:
# 🔹 2. Detect Stuck Vehicles
def detect_stuck_vehicle(df, params):
    """
    Detects stuck vehicles using a rolling window of speed and spatial consistency,
    segmented by trip_id to avoid cross-trip contamination.

    Parameters:
        df (pd.DataFrame): Must contain 'speed_mph', 'latitude', 'longitude', and 'trip_id'.
        params (dict): Widget-defined anomaly thresholds.

    Returns:
        pd.DataFrame: Flagged stuck vehicle anomalies.
    """
    speed_thresh = params.get("STUCK_SPEED_MPH", 1.0)   # mph
    window = params.get("STUCK_WINDOW_SIZE", 4)         # frames

    flagged_list = []

    for trip_id, group in df.groupby("trip_id"):
        group = group.sort_values("timestamp").copy()

        # Compute rolling conditions
        speed_ok = group["speed_mph"].rolling(window).mean() < speed_thresh
        lat_ok = group["latitude"].diff().abs().rolling(window).mean() < 0.0001
        lon_ok = group["longitude"].diff().abs().rolling(window).mean() < 0.0001

        stuck_flags = speed_ok & lat_ok & lon_ok

        flagged = group[stuck_flags.fillna(False)].copy()
        if not flagged.empty:
            flagged["anomaly_type"] = "stuck_vehicle"
            flagged_list.append(flagged)

    return pd.concat(flagged_list, ignore_index=True) if flagged_list else pd.DataFrame(columns=df.columns.tolist() + ["anomaly_type"])


In [ ]:
# # 🔹 3. Detect Impossible Speeds
# def detect_impossible_speeds(df, params):
#     """
#     Detects movement segments where computed speed exceeds threshold.

#     Parameters:
#         df (pd.DataFrame): Filtered or per-vehicle data
#         params (dict): Anomaly detection parameters from UI

#     Returns:
#         pd.DataFrame: Rows with flagged impossible-speed anomalies
#     """
#     # Convert mph to kph for geodesic-based calc
#     speed_limit_kph = params["SPEED_LIMIT_MPH"] * 1.60934

#     records = []
#     for i in range(1, len(df)):
#         row_prev, row_curr = df.iloc[i - 1], df.iloc[i]
#         time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()
#         if time_diff == 0:
#             continue

#         distance = geodesic(
#             (row_prev["latitude"], row_prev["longitude"]),
#             (row_curr["latitude"], row_curr["longitude"])
#         ).meters

#         speed_kph = (distance / time_diff) * 3.6  # m/s → km/h

#         if speed_kph > speed_limit_kph:
#             records.append({
#                 "vehicle_id": row_curr["vehicle_id"],
#                 "timestamp_curr": row_curr["timestamp"],
#                 "computed_speed_kph": speed_kph,
#                 "distance_m": distance,
#                 "anomaly_type": "impossible_speed"
#             })

#     return pd.DataFrame(records)


In [ ]:
# 🔹 3. Detect Impossible Speeds
def detect_impossible_speeds(df, params):
    """
    Detects movement segments where computed speed exceeds threshold,
    segmented by trip_id to avoid cross-trip errors.

    Parameters:
        df (pd.DataFrame): Must contain timestamp, lat/lon, trip_id
        params (dict): Anomaly detection thresholds

    Returns:
        pd.DataFrame: Flagged rows with 'impossible_speed'
    """
    speed_limit_kph = params["SPEED_LIMIT_MPH"] * 1.60934
    output_rows = []

    for trip_id, group in df.groupby("trip_id"):
        group = group.sort_values("timestamp").copy()

        for i in range(1, len(group)):
            row_prev, row_curr = group.iloc[i - 1], group.iloc[i]
            time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()
            if time_diff <= 0:
                continue

            distance = geodesic(
                (row_prev["latitude"], row_prev["longitude"]),
                (row_curr["latitude"], row_curr["longitude"])
            ).meters

            speed_kph = (distance / time_diff) * 3.6

            if speed_kph > speed_limit_kph:
                output_rows.append({
                    "vehicle_id": row_curr["vehicle_id"],
                    "trip_id": trip_id,
                    "timestamp_curr": row_curr["timestamp"],
                    "computed_speed_kph": speed_kph,
                    "distance_m": distance,
                    "anomaly_type": "impossible_speed"
                })

    return pd.DataFrame(output_rows)


In [ ]:
# # 🔹 4. Detect Backtracking (Heading Reversal)
# def detect_backtracking(df, params):
#     """
#     Detects heading reversals suggesting backtracking behavior.

#     Parameters:
#         df (pd.DataFrame): Filtered or per-vehicle data
#         params (dict): Anomaly detection parameters from UI

#     Returns:
#         pd.DataFrame: Rows where heading reversed beyond threshold
#     """
#     reversal_thresh = params["REVERSAL_HEADING_THRESHOLD"]

#     if "heading" not in df.columns:
#         return pd.DataFrame()

#     heading_diff = df["heading"].diff().abs()
#     backtrack_flags = heading_diff.between(reversal_thresh, 200)

#     return df[backtrack_flags.fillna(False)].assign(anomaly_type="backtracking")


In [ ]:
# 🔹 4. Detect Backtracking (Heading Reversal)
def detect_backtracking(df, params):
    """
    Detects heading reversals (backtracking) within trip boundaries only,
    with wraparound-safe angle diffing and traceability.

    Parameters:
        df (pd.DataFrame): Filtered per-vehicle data with heading + trip_id
        params (dict): Anomaly detection thresholds

    Returns:
        pd.DataFrame: Rows flagged as 'backtracking'
    """
    reversal_thresh = params.get("REVERSAL_HEADING_THRESHOLD", 120)
    output_frames = []

    if "heading" not in df.columns or "trip_id" not in df.columns:
        return pd.DataFrame()

    for trip_id, group in df.groupby("trip_id"):
        group = group.sort_values("timestamp").copy()

        # Compute heading difference with wraparound correction
        group["heading_prev"] = group["heading"].shift()
        group["heading_diff"] = (group["heading"] - group["heading_prev"]).abs()
        group["heading_diff"] = group["heading_diff"].apply(
            lambda x: min(x, 360 - x) if pd.notna(x) else x
        )

        # Identify backtracking
        backtrack_flags = group["heading_diff"].between(reversal_thresh, 200)

        flagged = group[backtrack_flags.fillna(False)].copy()
        flagged["anomaly_type"] = "backtracking"
        flagged["timestamp_prev"] = group["timestamp"].shift()

        output_frames.append(flagged)

    return pd.concat(output_frames, ignore_index=True) if output_frames else pd.DataFrame()



In [ ]:
# # 🔹 5. Detect Repeated Points
# def detect_repeated_points(df, params):
#     """
#     Detects repeated GPS coordinates (vehicle not moving).

#     Parameters:
#         df (pd.DataFrame): Filtered or per-vehicle snapshot data
#         params (dict): Anomaly detection parameters from UI

#     Returns:
#         pd.DataFrame: Rows flagged as having repeated lat/lon values
#     """
#     tolerance = abs(params.get("REPEATED_COORD_TOLERANCE", 1e-5))

#     if "latitude" not in df.columns or "longitude" not in df.columns:
#         return pd.DataFrame()

#     repeated = (
#         (df["latitude"].diff().abs() < tolerance) &
#         (df["longitude"].diff().abs() < tolerance)
#     )

#     return df[repeated.fillna(False)].assign(anomaly_type="repeated_points")


In [ ]:
# 🔹 5. Detect Repeated Points
def detect_repeated_points(df, params):
    """
    Detects repeated GPS coordinates (vehicle not moving), scoped to each trip.

    Parameters:
        df (pd.DataFrame): Per-vehicle GPS snapshot data
        params (dict): UI-provided anomaly detection thresholds

    Returns:
        pd.DataFrame: Rows flagged as 'repeated_points'
    """
    tolerance = abs(params.get("REPEATED_COORD_TOLERANCE", 1e-5))
    output = []

    if "latitude" not in df.columns or "longitude" not in df.columns or "trip_id" not in df.columns:
        return pd.DataFrame()

    for trip_id, group in df.groupby("trip_id"):
        group = group.sort_values("timestamp").copy()
        lat_repeat = group["latitude"].diff().abs() < tolerance
        lon_repeat = group["longitude"].diff().abs() < tolerance
        repeated = lat_repeat & lon_repeat

        flagged = group[repeated.fillna(False)].copy()
        flagged["anomaly_type"] = "repeated_points"
        output.append(flagged)

    return pd.concat(output, ignore_index=True) if output else pd.DataFrame()


In [ ]:
# # 🔹 6. Detect Disappearance Without Return
# def detect_disappeared(df, snapshot_end_time, params):
#     """
#     Detects vehicles that have not reappeared by the end of the snapshot period.

#     Parameters:
#         df (pd.DataFrame): Data for a single vehicle
#         snapshot_end_time (datetime): End of data collection
#         params (dict): Anomaly detection thresholds from widget

#     Returns:
#         pd.DataFrame: Single-row disappearance anomaly, or empty DataFrame
#     """
#     if df.empty:
#         return pd.DataFrame()

#     last_seen = df["timestamp"].max()
#     time_gap_sec = (snapshot_end_time - last_seen).total_seconds()
#     disappearance_thresh = params["DISAPPEARANCE_TIME_THRESHOLD"]

#     if time_gap_sec > disappearance_thresh:
#         return pd.DataFrame([{
#             "vehicle_id": df["vehicle_id"].iloc[0],
#             "last_seen": last_seen,
#             "anomaly_type": "disappearance"
#         }])

#     return pd.DataFrame()


In [ ]:
# 🔹 6. Detect Disappearance Without Return
def detect_disappeared(df, snapshot_end_time, params):
    """
    Detects disappearance per trip_id — if the trip ends and never resumes.

    Parameters:
        df (pd.DataFrame): Data for a single vehicle
        snapshot_end_time (datetime): End of snapshot
        params (dict): Anomaly config

    Returns:
        pd.DataFrame: One row per disappeared trip, if any
    """
    disappearance_thresh = params["DISAPPEARANCE_TIME_THRESHOLD"]
    output = []

    for trip_id, group in df.groupby("trip_id"):
        last_seen = group["timestamp"].max()
        time_gap_sec = (snapshot_end_time - last_seen).total_seconds()

        if time_gap_sec > disappearance_thresh:
            output.append({
                "vehicle_id": group["vehicle_id"].iloc[0],
                "trip_id": trip_id,
                "last_seen": last_seen,
                "anomaly_type": "disappearance"
            })

    return pd.DataFrame(output) if output else pd.DataFrame()


In [ ]:
# 🔹 7. Detect Early Appearance
def detect_early_appearance(df, snapshot_start_time, params):
    """
    Detects vehicles that appear too early (likely pre-start ghost data).

    Parameters:
        df (pd.DataFrame): Data for a single vehicle
        snapshot_start_time (datetime): Start of capture window
        params (dict): Anomaly detection thresholds from widget

    Returns:
        pd.DataFrame: Single-row anomaly, or empty DataFrame
    """
    margin_seconds = params.get("EARLY_APPEARANCE_MARGIN_SEC", 30)
    first_seen = df["timestamp"].min()
    delta = (first_seen - snapshot_start_time).total_seconds()

    if delta < margin_seconds:
        return pd.DataFrame([{
            "vehicle_id": df["vehicle_id"].iloc[0],
            "first_seen": first_seen,
            "anomaly_type": "early_appearance"
        }])
    return pd.DataFrame()


In [ ]:
# # 🔹 8. Detect Off-Route Movement
# def detect_off_route(df_vehicle, shape_lines, params):
#     """
#     Flags GPS points that are farther than OFF_ROUTE_BUFFER_M from the expected route shape.

#     Parameters:
#         df_vehicle (pd.DataFrame): All points for a single vehicle
#         shape_lines (dict): Dictionary of LineStrings per shape_id
#         params (dict): Thresholds including OFF_ROUTE_BUFFER_M

#     Returns:
#         pd.DataFrame: Off-route GPS points
#     """
#     buffer_m = params.get("OFF_ROUTE_BUFFER_M", 50)
#     records = []

#     for _, row in df_vehicle.iterrows():
#         shape_id = str(row.get("shape_id"))
#         if shape_id not in shape_lines:
#             continue  # shape not available

#         route_line = shape_lines[shape_id]
#         vehicle_point = Point(row["longitude"], row["latitude"])

#         # Project point onto shape line and compute geodesic distance
#         closest_point = route_line.interpolate(route_line.project(vehicle_point))
#         dist_m = geodesic(
#             (row["latitude"], row["longitude"]),
#             (closest_point.y, closest_point.x)
#         ).meters

#         if dist_m > buffer_m:
#             records.append({
#                 "vehicle_id": row["vehicle_id"],
#                 "timestamp": row["timestamp"],
#                 "route_short_name": row.get("route_short_name"),
#                 "distance_from_route_m": dist_m,
#                 "latitude": row["latitude"],
#                 "longitude": row["longitude"],
#                 "shape_id": shape_id,
#                 "anomaly_type": "off_route"
#             })

#     return pd.DataFrame(records)


In [ ]:
# 🔹 8. Detect Off-Route Movement
def detect_off_route(df_vehicle, shape_lines, params):
    """
    Detect off-route anomalies using trip-level shape validation.

    Parameters:
        df_vehicle (pd.DataFrame): All rows for a single vehicle.
        shape_lines (dict): shape_id → LineString
        params (dict): Dictionary with OFF_ROUTE_BUFFER_M

    Returns:
        pd.DataFrame: Flagged off-route points
    """
    buffer_m = params.get("OFF_ROUTE_BUFFER_M", 50)
    records = []

    for trip_id, trip_df in df_vehicle.groupby("trip_id"):
        # Check if all rows share the same shape_id
        shape_ids = trip_df["shape_id"].dropna().unique()

        if len(shape_ids) != 1:
            # Ambiguous shape_id — skip this trip to avoid misflagging
            continue

        shape_id = str(shape_ids[0])
        if shape_id not in shape_lines:
            continue

        route_line = shape_lines[shape_id]

        for _, row in trip_df.iterrows():
            vehicle_point = Point(row["longitude"], row["latitude"])
            projected = route_line.interpolate(route_line.project(vehicle_point))
            dist_m = geodesic(
                (row["latitude"], row["longitude"]),
                (projected.y, projected.x)
            ).meters

            if dist_m > buffer_m:
                records.append({
                    "vehicle_id": row["vehicle_id"],
                    "trip_id": trip_id,
                    "timestamp": row["timestamp"],
                    "route_short_name": row.get("route_short_name"),
                    "distance_from_route_m": dist_m,
                    "latitude": row["latitude"],
                    "longitude": row["longitude"],
                    "shape_id": shape_id,
                    "anomaly_type": "off_route"
                })

    return pd.DataFrame(records)

# Note: ✅ Key Protections Added
# Fix	Explanation
# groupby("trip_id")	Ensures anomalies only detected within a single trip context
# unique shape_id check	Filters out data where shape mapping is ambiguous
# skip trip if shape_id missing or mismatched	Prevents false positives from misassigned shapes
# Includes trip_id in output	Improves downstream traceability and debugging

In [ ]:
# Define options
detector_options = [
    ("Jumps & Gaps", "jumpgap"),
    ("Stuck Vehicles", "stuck"),
    ("Impossible Speeds", "speed"),
    ("Backtracking", "backtrack"),
    ("Repeated Points", "repeated"),
    ("Disappearance", "disappear"),
    ("Early Appearance", "early"),
    ("Off-Route", "offroute")
]

# Create selector widget
detector_selector = widgets.SelectMultiple(
    options=detector_options,
    value=[k for _, k in detector_options],  # default: all selected
    description="Detectors:",
    layout=widgets.Layout(width="60%")
)

# Create button
confirm_button = widgets.Button(
    description="Confirm Selection",
    button_style='primary',
    icon='check'
)

# Output area
selection_output = widgets.Output()

# Define callback
def on_confirm_clicked(b):
    with selection_output:
        clear_output()
        selected = list(detector_selector.value)
        print("✅ Selected anomaly detectors:")
        for detector in selected:
            print(f" - {detector}")
        # Optionally set a global variable for later use:
        global selected_detectors
        selected_detectors = selected

# Link button to callback
confirm_button.on_click(on_confirm_clicked)

# Display widgets
display(widgets.VBox([detector_selector, confirm_button, selection_output]))


In [ ]:
# # I believe we can skip step 1 because it was already done earlier
# # # Step 1: Normalize timestamp and rebuild vehicle groups
# # df_valid["timestamp"] = pd.to_datetime(df_valid["timestamp_collected"], utc=True)
# # vehicle_groups = dict(tuple(df_valid.groupby("vehicle_id")))

# # Step 2: Detect each anomaly type using widget-supplied parameters
# jumpgap_records = []
# stuck_records = []
# speed_records = []
# backtrack_records = []
# repeated_records = []
# disappear_records = []
# early_records = []
# offroute_records = []

# snapshot_start = df_valid["timestamp"].min()
# snapshot_end = df_valid["timestamp"].max()

# for vehicle_id, df_vehicle in vehicle_groups.items():
#     df_vehicle = df_vehicle.sort_values("timestamp").reset_index(drop=True)

#     # 1. Jumps and Gaps
#     jumpgap_records.extend(detect_jumps_and_gaps(df_vehicle, params=anomaly_params))

#     # 2. Stuck Vehicles
#     stuck = detect_stuck_vehicle(df_vehicle, params=anomaly_params)
#     if not stuck.empty:
#         stuck_records.append(stuck)

#     # 3. Impossible Speeds
#     speed = detect_impossible_speeds(df_vehicle, params=anomaly_params)
#     if not speed.empty:
#         speed_records.append(speed)

#     # 4. Backtracking
#     backtrack = detect_backtracking(df_vehicle, params=anomaly_params)
#     if not backtrack.empty:
#         backtrack_records.append(backtrack)

#     # 5. Repeated Points
#     repeat = detect_repeated_points(df_vehicle, params=anomaly_params)
#     if not repeat.empty:
#         repeated_records.append(repeat)

#     # 6. Disappearance
#     disappear = detect_disappeared(df_vehicle, snapshot_end_time=snapshot_end, params=anomaly_params)
#     if not disappear.empty:
#         disappear_records.append(disappear)

#     # 7. Early Appearance
#     early = detect_early_appearance(df_vehicle, snapshot_start_time=snapshot_start, params=anomaly_params)
#     if not early.empty:
#         early_records.append(early)

#     # 8. Off-Route
#     offroute = detect_off_route(df_vehicle, shape_lines=shape_lines, params=anomaly_params)
#     if not offroute.empty:
#         offroute_records.append(offroute)

# # Step 3: Combine to DataFrames
# df_anomalies_jumpgap   = pd.DataFrame(jumpgap_records)
# df_anomalies_stuck     = pd.concat(stuck_records, ignore_index=True)     if stuck_records     else pd.DataFrame()
# df_anomalies_speed     = pd.concat(speed_records, ignore_index=True)     if speed_records     else pd.DataFrame()
# df_anomalies_backtrack = pd.concat(backtrack_records, ignore_index=True) if backtrack_records else pd.DataFrame()
# df_anomalies_repeated  = pd.concat(repeated_records, ignore_index=True)  if repeated_records  else pd.DataFrame()
# df_anomalies_disappear = pd.concat(disappear_records, ignore_index=True) if disappear_records else pd.DataFrame()
# df_anomalies_early     = pd.concat(early_records, ignore_index=True)     if early_records     else pd.DataFrame()
# df_anomalies_offroute  = pd.concat(offroute_records, ignore_index=True)  if offroute_records  else pd.DataFrame()

# # Step 4: Combine all anomalies into a single DataFrame
# anomaly_frames = [
#     df_anomalies_jumpgap,
#     df_anomalies_stuck,
#     df_anomalies_speed,
#     df_anomalies_backtrack,
#     df_anomalies_repeated,
#     df_anomalies_disappear,
#     df_anomalies_early,
#     df_anomalies_offroute
# ]

# anomaly_frames = [df for df in anomaly_frames if 'anomaly_type' in df.columns and not df.empty]
# df_anomalies_full = pd.concat(anomaly_frames, ignore_index=True)

# # Report
# print("✅ Unified anomaly count:", len(df_anomalies_full))
# display(df_anomalies_full["anomaly_type"].value_counts())


In [ ]:
# Selected detectors from UI
selected_detectors = list(detector_selector.value)

# Reset result containers
jumpgap_records = []
stuck_records = []
speed_records = []
backtrack_records = []
repeated_records = []
disappear_records = []
early_records = []
offroute_records = []

snapshot_start = df_valid["timestamp"].min()
snapshot_end = df_valid["timestamp"].max()

# Run detectors conditionally
for vehicle_id, df_vehicle in vehicle_groups.items():
    df_vehicle = df_vehicle.sort_values("timestamp").reset_index(drop=True)

    if "jumpgap" in selected_detectors:
        jumpgap_records.extend(detect_jumps_and_gaps(df_vehicle, params=anomaly_params))

    if "stuck" in selected_detectors:
        stuck = detect_stuck_vehicle(df_vehicle, params=anomaly_params)
        if not stuck.empty:
            stuck_records.append(stuck)

    if "speed" in selected_detectors:
        speed = detect_impossible_speeds(df_vehicle, params=anomaly_params)
        if not speed.empty:
            speed_records.append(speed)

    if "backtrack" in selected_detectors:
        backtrack = detect_backtracking(df_vehicle, params=anomaly_params)
        if not backtrack.empty:
            backtrack_records.append(backtrack)

    if "repeated" in selected_detectors:
        repeat = detect_repeated_points(df_vehicle, params=anomaly_params)
        if not repeat.empty:
            repeated_records.append(repeat)

    if "disappear" in selected_detectors:
        disappear = detect_disappeared(df_vehicle, snapshot_end_time=snapshot_end, params=anomaly_params)
        if not disappear.empty:
            disappear_records.append(disappear)

    if "early" in selected_detectors:
        early = detect_early_appearance(df_vehicle, snapshot_start_time=snapshot_start, params=anomaly_params)
        if not early.empty:
            early_records.append(early)

    if "offroute" in selected_detectors:
        offroute = detect_off_route(df_vehicle, shape_lines=shape_lines, params=anomaly_params)
        if not offroute.empty:
            offroute_records.append(offroute)

# Step 3: Combine to DataFrames
df_anomalies_jumpgap   = pd.DataFrame(jumpgap_records)
df_anomalies_stuck     = pd.concat(stuck_records, ignore_index=True)     if stuck_records     else pd.DataFrame()
df_anomalies_speed     = pd.concat(speed_records, ignore_index=True)     if speed_records     else pd.DataFrame()
df_anomalies_backtrack = pd.concat(backtrack_records, ignore_index=True) if backtrack_records else pd.DataFrame()
df_anomalies_repeated  = pd.concat(repeated_records, ignore_index=True)  if repeated_records  else pd.DataFrame()
df_anomalies_disappear = pd.concat(disappear_records, ignore_index=True) if disappear_records else pd.DataFrame()
df_anomalies_early     = pd.concat(early_records, ignore_index=True)     if early_records     else pd.DataFrame()
df_anomalies_offroute  = pd.concat(offroute_records, ignore_index=True)  if offroute_records  else pd.DataFrame()

# Step 4: Combine all selected anomalies
anomaly_frames = []
if "jumpgap" in selected_detectors:   anomaly_frames.append(df_anomalies_jumpgap)
if "stuck" in selected_detectors:     anomaly_frames.append(df_anomalies_stuck)
if "speed" in selected_detectors:     anomaly_frames.append(df_anomalies_speed)
if "backtrack" in selected_detectors: anomaly_frames.append(df_anomalies_backtrack)
if "repeated" in selected_detectors:  anomaly_frames.append(df_anomalies_repeated)
if "disappear" in selected_detectors: anomaly_frames.append(df_anomalies_disappear)
if "early" in selected_detectors:     anomaly_frames.append(df_anomalies_early)
if "offroute" in selected_detectors:  anomaly_frames.append(df_anomalies_offroute)

anomaly_frames = [df for df in anomaly_frames if 'anomaly_type' in df.columns and not df.empty]
df_anomalies_full = pd.concat(anomaly_frames, ignore_index=True) if anomaly_frames else pd.DataFrame()

# Report
print("Anomaly count (pre-dedup):", len(df_anomalies_full))
df_anomalies_full.drop_duplicates(inplace=True)
print("Anomaly count (post-dedup):", len(df_anomalies_full))

print("✅ Unified anomaly count:", len(df_anomalies_full))
if not df_anomalies_full.empty:
    display(df_anomalies_full["anomaly_type"].value_counts())
else:
    print("⚠️ No anomalies detected.")


In [ ]:
df_anomalies_full.describe(include='all')

In [ ]:
df_anomalies_full.head(100)

In [ ]:
# print("Total rows:", len(df_anomalies_full))
# print("Unique rows:", len(df_anomalies_full.drop_duplicates()))
# print("Duplicate rows:", df_anomalies_full.duplicated().sum())


####Merge check
2. Shape-Based Merge into df_anomalies_offroute
- df_anomalies_offroute = df_anomalies_offroute.merge(
    shape_route_map, on="shape_id", how="left", suffixes=("", "_from_map")
)

🔍 Key issue: This assumes that df_anomalies_offroute has an accurate shape_id per row.

But this is not always valid, because:

- Anomalies are grouped by vehicle_id and timestamp.

- There's no explicit trip_id or shape_id preserved per anomaly.

- If a single vehicle changes shapes mid-route, this could assign the wrong route.

In [ ]:
shape_route_map.head()

In [ ]:
shape_route_map.info()

In [ ]:
df_anomalies_offroute.info()

In [ ]:
df_anomalies_offroute['shape_id'].value_counts()

In [ ]:
# Ensure shape_id is a column (not index)
shape_route_map = shape_route_map.reset_index()

# Ensure both shape_id columns are string type before merge
df_anomalies_offroute["shape_id"] = df_anomalies_offroute["shape_id"].astype(str)
shape_route_map["shape_id"] = shape_route_map["shape_id"].astype(str)

# Join to enrich anomalies with route info
df_anomalies_offroute = (
    df_anomalies_offroute
    .merge(shape_route_map, on="shape_id", how="left", suffixes=("", "_from_map"))
)

# # Patch missing route names
# df_anomalies_offroute["route_short_name"] = (
#     df_anomalies_offroute["route_short_name"]
#     .fillna(df_anomalies_offroute["route_short_name_from_map"])
# )

# # Clean up
# df_anomalies_offroute = df_anomalies_offroute.drop(columns=["route_short_name_from_map"])


In [ ]:
df_anomalies_offroute.describe(include='all')

In [ ]:
df_anomalies_offroute.isna().sum()

In [ ]:
#Sanity Check
missing_routes = df_anomalies_offroute["route_short_name"].isnull().sum()
print(f"Remaining anomalies with missing route_short_name: {missing_routes}")
#If this prints 0, you’ve successfully patched all entries.

if missing_routes > 0:
    print("Sample of anomalies with missing route_short_name:")
    display(df_anomalies_offroute[df_anomalies_offroute["route_short_name"].isnull()].head())


In [ ]:
df_full.isna().sum()

In [ ]:
df_full.columns

#####Debugging note: I believe the cell below may be creating some problems

In [ ]:
df_full['data_quality'].value_counts()

In [ ]:
#🔹 1.1 Filter for Valid Rows
df_valid = df_full[df_full["data_quality"] == "valid"].copy()

#🔹 1.2 Parse Timestamps
df_valid["timestamp"] = pd.to_datetime(df_valid["timestamp_collected"], utc=True)

#🔹 1.3 Sort by Vehicle and Timestamp
df_valid = df_valid.sort_values(by=["vehicle_id", "timestamp"])

#🔹 1.4 Organize by Vehicle
#This creates a dictionary keyed by vehicle ID, each with a sorted DataFrame:
vehicle_groups = dict(tuple(df_valid.groupby("vehicle_id")))

#You can confirm how many distinct vehicles you’re tracking:
print("Vehicle count:", len(vehicle_groups))


#Summary Stats by Route/Anomaly

####Debugging check: possible problem here with route_lookup

In [ ]:
# df_valid.head(20)

In [ ]:
# df_valid.columns

In [ ]:
# routes.columns

In [ ]:
# trips.columns

In [ ]:
# Build trip-to-route lookup from GTFS static data
trip_to_route_lookup = trips[["trip_id", "route_id"]].merge(
    routes[["route_id", "route_short_name"]], on="route_id", how="left"
)

In [ ]:
trip_to_route_lookup.columns

In [ ]:
trip_to_route_lookup['route_short_name'].value_counts()

In [ ]:
trip_to_route_lookup['route_id'].value_counts()

In [ ]:
# Build trip-to-route lookup from GTFS static data
trip_to_route_lookup = trips[["trip_id", "route_id"]].merge(
    routes[["route_id", "route_short_name"]], on="route_id", how="left"
)

# Ensure trip_id types match
df_valid["trip_id"] = df_valid["trip_id"].astype(str)
trip_to_route_lookup["trip_id"] = trip_to_route_lookup["trip_id"].astype(str)

#I believe this may have been resolved by splitting route_short_name into x and y
# # Drop route_short_name from df_valid if it exists to avoid merge collision
# if "route_short_name" in df_valid.columns:
#     df_valid = df_valid.drop(columns=["route_short_name"])

trip_to_route_lookup["route_id_trips"] = trip_to_route_lookup["route_id"].copy()
trip_to_route_lookup["route_short_name_trips"] = trip_to_route_lookup["route_short_name"].copy()


# Merge to patch in route_short_name
df_valid = df_valid.merge(trip_to_route_lookup[["trip_id", "route_id_trips", "route_short_name_trips"]], on="trip_id", how="left")


In [ ]:
df_valid.columns

In [ ]:
df_valid.head(10)

In [ ]:
#Step 1: Build a lookup table
route_lookup = df_valid[["vehicle_id", "timestamp", "route_short_name"]].copy()
route_lookup["timestamp"] = pd.to_datetime(route_lookup["timestamp"], utc=True)


In [ ]:
df_anomalies_full.info()

In [ ]:
#I believe this is already resolved
# #Step 2: Also convert timestamp in anomalies to datetime
# df_anomalies_full["timestamp"] = pd.to_datetime(df_anomalies_full["timestamp"], utc=True)


In [ ]:
#Create a backup so we don't have to keep rerunning everything
df_anomalies_full_copy = df_anomalies_full.copy()

# #Rebuild from scratch:

# df_anomalies_full_backup = pd.concat([
    # df_anomalies_jumpgap,
    # df_anomalies_stuck,
    # df_anomalies_speed,
    # df_anomalies_backtrack,
    # df_anomalies_repeated,
    # df_anomalies_disappear,
    # df_anomalies_early,
    # df_anomalies_offroute
# ], ignore_index=True)

In [ ]:
# df_anomalies_full = df_anomalies_full_copy

✅ df_anomalies_full_copy has the correct anomaly types:
repeated_points     46200  
stuck_vehicle       30988  
jump_or_gap          9200  ← LOST  
off_route            6767  
backtracking         1616  
impossible_speed      434  ← LOST  
disappearance          42  ← LOST  
early_appearance        1  ← LOST  
But after the route name merge, your current df_anomalies_full is missing the last four types.


####Merge check
(Note: this was resolved in .v8)
3. merge_asof Join Between df_anomalies_full and route_lookup
- merge_result = pd.merge_asof(
    df_anomalies_full.sort_values("timestamp"),
    route_lookup.sort_values("timestamp"),
    on="timestamp",
    by="vehicle_id",
    direction="nearest",
    ...
)

🔍 Potential source of corruption:
- If route_lookup (used to get route IDs for vehicles) was built without tight trip_id / shape_id constraints, it can erroneously associate the wrong route to a vehicle.
- merge_asof can assign the nearest route in time — but this may be a different shape or trip.

🛑 This is a prime suspect for inflated off_route counts.

In [ ]:
#Part of the merge investigation; can be removed later
# # Check how many unique routes are assigned per vehicle
# route_counts = df_anomalies_full.groupby("vehicle_id")["route_short_name"].nunique()
# conflict_vehicles = route_counts[route_counts > 1].index

# print(f"🚨 Vehicles with conflicting routes: {len(conflict_vehicles)}")


In [ ]:
# #Diagnostics Step 1: See how many non-null entries are already in merge_subset["route_short_name"]
# print("🔎 Pre-merge route_short_name stats in merge_subset:")
# print(merge_subset["route_short_name"].value_counts(dropna=False).head(10))
# print("Total non-null route_short_name:", merge_subset["route_short_name"].notna().sum())


In [ ]:
# #Diagnostics Step 2: Perform the exact merge but using a temporary suffix to compare side-by-side
# merged_temp = pd.merge(
#     merge_subset,
#     route_lookup_clean[["vehicle_id", "timestamp", "route_short_name"]],
#     on=["vehicle_id", "timestamp"],
#     how="left",
#     suffixes=("_original", "_lookup")
# )


In [ ]:
# #Diagnostics Step 3: Check how often the two columns agree, disagree, or one is missing
# both_present = merged_temp["route_short_name_original"].notna() & merged_temp["route_short_name_lookup"].notna()
# same = merged_temp["route_short_name_original"] == merged_temp["route_short_name_lookup"]
# disagree = both_present & ~same

# print("✅ Both present:", both_present.sum())
# print("✅ Agreeing values:", (same & both_present).sum())
# print("❌ Disagreeing values:", disagree.sum())


In [ ]:
# #Diagnostics Step 4: View some of the disagreements
# print("⚠️ Sample rows where route_short_name disagrees:")
# display(merged_temp.loc[disagree, ["vehicle_id", "timestamp", "route_short_name_original", "route_short_name_lookup"]].head(10))


In [ ]:
# #Diagnostics Step 5: Inspect which source is more complete
# print("Original non-null:", merged_temp["route_short_name_original"].notna().sum())
# print("Lookup non-null:", merged_temp["route_short_name_lookup"].notna().sum())


In [ ]:
df_anomalies_full.head()

In [ ]:
#Let's redo the merging *.v8.2
#Actually... not sure why we were doing this at all
#This is from *.v8 addition, basically imposing exact matches on the merge, opposed to an earlier version that would use merge_asof and perform nearest matching
# 🧼 Step 1: Clean merge inputs
valid_rows = df_anomalies_full["timestamp"].notna()
merge_subset = df_anomalies_full.loc[valid_rows].copy()
route_lookup_clean = route_lookup[route_lookup["timestamp"].notna()].copy()

# 🔧 Ensure matching dtypes
merge_subset["vehicle_id"] = merge_subset["vehicle_id"].astype(str)
route_lookup_clean["vehicle_id"] = route_lookup_clean["vehicle_id"].astype(str)

In [ ]:
merge_subset.columns

In [ ]:
route_lookup_clean.columns

In [ ]:
# merge_redux = merge_subset.merge(route_lookup_clean[['vehicle_id', 'timestamp']])

In [ ]:
# #This is the *.v8 addition, basically imposing exact matches on the merge, opposed to an earlier version that would use merge_asof and perform nearest matching
# # 🧼 Step 1: Clean merge inputs
# valid_rows = df_anomalies_full["timestamp"].notna()
# merge_subset = df_anomalies_full.loc[valid_rows].copy()
# route_lookup_clean = route_lookup[route_lookup["timestamp"].notna()].copy()

# # 🔧 Ensure matching dtypes
# merge_subset["vehicle_id"] = merge_subset["vehicle_id"].astype(str)
# route_lookup_clean["vehicle_id"] = route_lookup_clean["vehicle_id"].astype(str)

# # ✅ Drop empty original route column
# merge_subset = merge_subset.drop(columns=["route_short_name"], errors="ignore")

# # 🔁 Step 2: Exact merge
# merged = pd.merge(
#     merge_subset,
#     route_lookup_clean[["vehicle_id", "timestamp", "route_short_name"]],
#     on=["vehicle_id", "timestamp"],
#     how="left"
# )

# # 🔁 Step 3: Patch back into df_anomalies_full
# route_name_map = merged.set_index(merge_subset.index)["route_short_name"]
# df_anomalies_full["route_short_name"] = df_anomalies_full["route_short_name"].combine_first(route_name_map)

# # ✅ Step 4: Final check
# print("✅ Remaining missing route names:", df_anomalies_full["route_short_name"].isna().sum())


#####*.v8.2 250731-1630 note: not sure why we were cleaning the data above
and I think this merging may have caused some issues, so skipping it for now

In [ ]:
df_anomalies_full[df_anomalies_full["route_short_name"].isna()].sample(5)


In [ ]:
df_anomalies_full.isna().sum()

##Debugging waypoint 2507251634

In [ ]:
df_anomalies_full['route_short_name'].value_counts()

In [ ]:
#Presently there are no unknowns or nulls, so we don't have to worry about this
# # Treat 'Unknown' as null
# df_anomalies_full["route_short_name"] = df_anomalies_full["route_short_name"].replace("Unknown", pd.NA)

# # Only apply fallback if the lookup column is still present
# if "route_short_name_from_lookup" in df_anomalies_full.columns:
#     df_anomalies_full["route_short_name"] = df_anomalies_full["route_short_name"].combine_first(
#         df_anomalies_full["route_short_name_from_lookup"]
#     )

# # Confirm
# print("Remaining missing route names:", df_anomalies_full["route_short_name"].isna().sum())
# print(df_anomalies_full["route_short_name"].value_counts(dropna=False).head())


In [ ]:
#Same as above cell
# # Confirm null replacement and summary
# df_anomalies_full["route_short_name"] = df_anomalies_full["route_short_name"].replace("Unknown", pd.NA)

# # Summary diagnostics
# print("Remaining missing route names:", df_anomalies_full["route_short_name"].isna().sum())
# print(df_anomalies_full["route_short_name"].value_counts(dropna=False).head())


In [ ]:
print("Checkpoint anomaly type breakdown:")
print(df_anomalies_full["anomaly_type"].value_counts(dropna=False))

print("Checkpoint anomaly type breakdown (copy):")
print(df_anomalies_full_copy["anomaly_type"].value_counts(dropna=False))

In [ ]:
print("Unique types in anomaly_type:")
print(set(type(val) for val in df_anomalies_full["anomaly_type"].unique()))


In [ ]:
print("Full anomaly types (raw):", df_anomalies_full["anomaly_type"].unique())
print("Full anomaly types (with types):", [(val, type(val)) for val in df_anomalies_full["anomaly_type"].unique()])


In [ ]:
print(selected_detectors)

In [ ]:
df_anomalies_full["anomaly_type"].unique()

In [ ]:
# Step 1: Clean anomaly type column
df_temp = df_anomalies_full.copy()
df_temp["route_short_name"] = df_temp["route_short_name"].fillna("(Missing)")
df_temp["anomaly_type_clean"] = df_temp["anomaly_type"].astype(str)

# Step 2: Build pivot table
pivot = df_temp.pivot_table(
    index="route_short_name",
    columns="anomaly_type_clean",
    aggfunc="size",
    fill_value=0
)

# Step 3: Manually reindex using the full list
expected_columns = [
    "repeated_points", "stuck_vehicle", "jump_or_gap", "off_route",
    "backtracking", "impossible_speed", "disappearance", "early_appearance"
]
pivot = pivot.reindex(columns=expected_columns, fill_value=0)

# Step 4: Diagnostics
print("✅ Total in pivot:", pivot.sum().sum())
print("✅ Total in df:", df_anomalies_full["anomaly_type"].notna().sum())
print("✅ Columns in pivot:", pivot.columns.tolist())


Key Fix Summary (for documentation or future review):
- Problem cause: pivot_table(...).reindex(columns=sorted(...)) introduced TypeError due to sorting mixed types (possibly hidden NaNs or object-type weirdness in internal index representation).

- Confirmed all anomaly_type values were str, but reindexing still failed — indicating a more subtle index-level mismatch.

Solution: Explicitly cast anomaly_type to string and reindex with a predefined list of expected columns.

In [ ]:
pivot

In [ ]:
#Second pivot table to try looking at things other than route_short_name
# Step 1: Clean anomaly type column
df_temp2 = df_anomalies_full.copy()
df_temp2["trip_id"] = df_temp2["trip_id"].fillna("(Missing)")
df_temp2["anomaly_type_clean"] = df_temp2["anomaly_type"].astype(str)

# Step 2: Build pivot table
pivot2 = df_temp2.pivot_table(
    index="trip_id",
    columns="anomaly_type_clean",
    aggfunc="size",
    fill_value=0
)

# Step 3: Manually reindex using the full list
expected_columns = [
    "repeated_points", "stuck_vehicle", "jump_or_gap", "off_route",
    "backtracking", "impossible_speed", "disappearance", "early_appearance"
]
pivot2 = pivot2.reindex(columns=expected_columns, fill_value=0)

# Step 4: Diagnostics
print("✅ Total in pivot:", pivot2.sum().sum())
print("✅ Total in df:", df_anomalies_full["anomaly_type"].notna().sum())
print("✅ Columns in pivot:", pivot2.columns.tolist())

In [ ]:
pivot2

####Debuggin in *.v8, to do:
Thanks for the clear context and diagnostic output — this is an important signal, and you're absolutely right to question it.

---

### 🔍 What's Going Wrong

The anomaly types:

* `"jump_or_gap"`, `"impossible_speed"`, `"disappearance"`, `"early_appearance"`

are **only appearing under `(Missing)` routes**, while all known route numbers (like `1`, `16`, `66`) show `0` in those columns.

Yet we know these anomaly types **should be assigned to real routes** in many cases.

---

### 🧠 Root Cause

Those anomalies **likely didn't get their `route_short_name` correctly assigned** during the earlier merge. Here's why:

* Many of these anomaly types are generated from **timestamp-level snapshot comparisons**, often between vehicles’ **first or last records**, or **with subtle time gaps**.
* If the timestamp didn’t match exactly with a route in the `route_lookup`, and you removed `merge_asof`, they **weren’t matched at all**.
* So they fell through with missing `route_short_name`, and got lumped into the `"(Missing)"` category in the pivot.

In short:

> You now get only **exact route matches**, which means **no match = Missing = misattributed anomalies**.

---

### ✅ Recommended Fix

To preserve strict merge logic *but still capture route info for jump/disappearance-type anomalies*, you need a **separate merge strategy** for those rows. Here's how:

---

#### 🔄 Updated Strategy

1. **Retain your current exact-match logic** for most anomalies (good for accuracy).
2. **For rows that still have no route**, use a **second fallback join**:

   * Join on `(vehicle_id, rounded timestamp)` with small ± tolerance via *manual logic*, **not `merge_asof`**.
   * Or pre-fill route for vehicles known to follow the same route across time.

Alternatively, if you're okay adding a **hybrid merge strategy**, then:

* First: `merge` (strict)
* Then: `merge_asof` only on rows that are still `NaN` for route
* Flag those rows as `fallback_match = True` for auditability

---

### 🚧 Immediate Diagnostic You Can Try

Let’s confirm our theory first.

Run this:

```python
missing_routes_df = df_anomalies_full[df_anomalies_full["route_short_name"].isna()]
print("🚫 Missing route entries by anomaly type:")
print(missing_routes_df["anomaly_type"].value_counts())
```

You’ll probably see that all the `jump_or_gap`, `impossible_speed`, etc. live in this `missing_routes_df`.

---

### 🛠️ Optional Patch to Recover These

If you’re willing to use a **limited scope fallback merge**, try this:

```python
# Find rows that still have missing route
missing_route_rows = df_anomalies_full["route_short_name"].isna()

# Attempt fallback merge on these using merge_asof (only here)
fallback_merge = pd.merge_asof(
    df_anomalies_full.loc[missing_route_rows].sort_values("timestamp"),
    route_lookup_clean.sort_values("timestamp"),
    on="timestamp",
    by="vehicle_id",
    direction="nearest",
    tolerance=pd.Timedelta("60s")
)

# Patch only where it's still missing
df_anomalies_full.loc[missing_route_rows, "route_short_name"] = fallback_merge["route_short_name"]
```

⚠️ This preserves your integrity for the main dataset, and **only relaxes the constraint for those otherwise unclassified rows**.

---

Would you like help implementing that fallback merge and visual diagnostic?


** Note: this was debugging notes from ≤*.v6 **

Interestingly, the (Missing) route_short_name has the only instances of jump_or_gap, impossible_speed, disappearance, early_appearance


That’s a critical diagnostic insight — and it likely explains why those anomaly types kept disappearing in earlier pivot tables:

🔍 Root Cause:
By default, pivot_table uses only the combination of index and columns values that exist. If all rows for some anomaly types are associated with a missing route_short_name (NaN), and you drop or exclude these during preprocessing, those types vanish from the pivot.

When you did:

df["route_short_name"] = df["route_short_name"].fillna("(Missing)")

You ensured that:

Those rows were not excluded from the index.

Their anomaly types were included and preserved in the pivot.

✅ Implications:
You must preserve (Missing) rows if you want a complete count across all anomaly types.

If you filter by route, or use .dropna() on route_short_name, anomaly types with only missing routes will be silently excluded.

When you view a pivot per route, types like jump_or_gap, impossible_speed, and disappearance may seem to “disappear” — but they’re actually only found in the (Missing) bucket.

---
To ensure full awareness in reporting or visualizing these anomaly types, do one of the following:

Always include a (Missing) category in route-level pivots.

Add a diagnostic count like:


In [ ]:
print("Anomalies with missing route names by type:")
print(df_anomalies_full[df_anomalies_full["route_short_name"].isna()]["anomaly_type"].value_counts())


In [ ]:
df_anomalies_full["anomaly_type"].value_counts(dropna=False)


#Plot in Folium

In [ ]:
#I don't think this is actually useful at this point
# # Step 1: Create a working copy
# df_plot_base = df_anomalies_full.copy()

# # Step 2: Drop rows with missing coordinates or timestamps
# df_plot_base = df_plot_base.dropna(subset=["latitude", "longitude", "timestamp_collected"])

# # Step 3: Ensure coordinate and timestamp types are correct
# df_plot_base["latitude"] = pd.to_numeric(df_plot_base["latitude"], errors="coerce")
# df_plot_base["longitude"] = pd.to_numeric(df_plot_base["longitude"], errors="coerce")
# df_plot_base["timestamp_collected"] = pd.to_datetime(df_plot_base["timestamp_collected"], errors="coerce", utc=True)

# # Step 4: Optional — Filter for anomalies with known route_short_name
# df_plot_base = df_plot_base[df_plot_base["route_short_name"].notna()]

# # Step 5: Optional — Save a filtered subset by anomaly type, route, or vehicle for visualization
# # You can update these filters dynamically for an interactive tool
# selected_anomaly_type = "stuck_vehicle"
# selected_route = "66"
# # Example: match on anomaly type and route
# df_plot_filtered = df_plot_base[
#     (df_plot_base["anomaly_type"] == selected_anomaly_type) &
#     (df_plot_base["route_short_name"] == selected_route)
# ]

# # Preview result
# print(f"🗺️ Plotting subset for anomaly: '{selected_anomaly_type}' on route '{selected_route}'")
# print(df_plot_filtered[["vehicle_id", "timestamp_collected", "latitude", "longitude", "anomaly_type"]].head())


In [ ]:
# Use full anomaly dataframe
df_full2 = df_anomalies_full.copy()

# Ensure lat/lon are floats
df_full2["latitude"] = pd.to_numeric(df_full2["latitude"], errors="coerce")
df_full2["longitude"] = pd.to_numeric(df_full2["longitude"], errors="coerce")

# Ensure timestamp_collected is datetime
df_full2["timestamp_collected"] = pd.to_datetime(df_full2["timestamp_collected"], errors="coerce", utc=True)

# Compute diffs for position and time
df_full2["lat_diff"] = df_full2.groupby("vehicle_id")["latitude"].diff()
df_full2["lon_diff"] = df_full2.groupby("vehicle_id")["longitude"].diff()
df_full2["jump_dist"] = (df_full2["lat_diff"]**2 + df_full2["lon_diff"]**2)**0.5
df_full2["time_diff"] = df_full2.groupby("vehicle_id")["timestamp_collected"].diff().dt.total_seconds()

# Label jump and disappearance events using anomaly_params
df_full2["is_jump"] = df_full2["jump_dist"] > anomaly_params["JUMP_DISTANCE_THRESHOLD"]
df_full2["is_disappearance"] = df_full2["time_diff"] > anomaly_params["DISAPPEARANCE_TIME_THRESHOLD"]

# Create enriched jump DataFrame
jumps_df = df_full2[df_full2["is_jump"]].copy()
jumps_df["lat_prev"] = df_full2.groupby("vehicle_id")["latitude"].shift()
jumps_df["lon_prev"] = df_full2.groupby("vehicle_id")["longitude"].shift()
jumps_df["timestamp_prev"] = df_full2.groupby("vehicle_id")["timestamp_collected"].shift()
jumps_df["timestamp_curr"] = jumps_df["timestamp_collected"]

# Filter to vehicles with enough jumps
jump_counts = jumps_df["vehicle_id"].value_counts()
keep_jumpers = jump_counts[jump_counts >= anomaly_params["MIN_JUMP_COUNT_PER_VEHICLE"]].index
jumps_df = jumps_df[jumps_df["vehicle_id"].isin(keep_jumpers)]

# Base frame of jumpers for disappearance/reappearance
df_jumpers_only = df_full2[df_full2["vehicle_id"].isin(keep_jumpers)].copy()

# Identify disappearance and reappearance points
disappear_df = df_jumpers_only[df_jumpers_only["is_disappearance"]].copy()
is_reappear = df_jumpers_only["is_disappearance"].shift(-1).fillna(False)
reappear_df = df_jumpers_only[is_reappear].copy()

# Summary of Key Changes:
# ✅ Replaces JUMP_DISTANCE_THRESHOLD, DISAPPEARANCE_TIME_THRESHOLD, and MIN_JUMP_COUNT_PER_VEHICLE with their anomaly_params equivalents.
# ✅ Maintains existing structure and logic.
# ✅ Compatible with all downstream Folium plotting layers.


In [ ]:
#I believe this can be depricated
# # that traceback confirms the immediate issue: after merging into shapes_df, the column route_short_name is not present, likely due to:

# # A failed merge (i.e. no matching shape_id values across shapes_df and shape_route_map).

# # The column being named something else (e.g. a merge conflict that renamed it, or a typo).

# # An earlier shapes_df overwrite or clean-up step that removed key columns.

# print("🔍 Columns in shapes_df:", shapes_df.columns.tolist())

# # Check if shape_route_map exists and contains route_short_name
# if 'shape_route_map' in globals():
#     print("✅ Columns in shape_route_map:", shape_route_map.columns.tolist())
#     print("✅ Sample shape_route_map rows:")
#     display(shape_route_map.head())
# else:
#     print("❌ shape_route_map does not exist")

# # Check shape_id overlap
# if 'shapes_df' in globals():
#     print("🧪 shape_id in shapes_df:", shapes_df["shape_id"].nunique())
#     print("🧪 shape_id in shape_route_map:", shape_route_map["shape_id"].nunique())
#     print("🧪 Overlapping shape_ids:", len(set(shapes_df["shape_id"]).intersection(shape_route_map["shape_id"])))


In [ ]:
#Step 1: Extract shape_id to route_id from trips_df
shape_to_route = trips[["shape_id", "route_id"]].drop_duplicates()
# trips.txt defines which route uses which shape.
# shape_to_route gives a clean 1:1 or 1:many mapping.

#Step 2: Map route_id to route_short_name from routes_df
route_id_to_name = routes[["route_id", "route_short_name"]]
#Brings in human-readable route names like "1", "10", "66".

#Step 3: Merge to associate shape_id with route_short_name
shape_route_map = shape_to_route.merge(route_id_to_name, on="route_id", how="left")
#You now have a table with: shape_id, route_id, and route_short_name

shapes_copy = shapes.copy()

# Step 4: Drop preexisting route_short_name to avoid merge conflicts
if "route_short_name" in shapes_copy.columns:
    shapes_copy.drop(columns=["route_short_name"], inplace=True)

# Step 4 (retry): Merge into shapes_df
shapes_df = shapes_copy.merge(shape_route_map, on="shape_id", how="left")

# Step 5: Confirm structure
assert shapes_df["route_short_name"].notna().all(), "❌ Some shapes are missing route names!"
print("✅ Final columns in shapes_df:", shapes_df.columns.tolist())
display(shapes_df[["shape_id", "route_short_name"]].drop_duplicates().head())





In [ ]:
#I believe this can be depricated
# # Next Step: Clean up unnecessary suffix columns
# # Since the final route_short_name column is already correct, and we don’t need the duplicated *_x and *_y columns, you can safely drop them:
# # Clean up leftover duplicate columns
# cols_to_drop = [col for col in shapes_df.columns if col.endswith("_x") or col.endswith("_y")]
# shapes_df.drop(columns=cols_to_drop, inplace=True)

# # Final confirmation
# print("✅ Cleaned columns in shapes_df:", shapes_df.columns.tolist())
# display(shapes_df[["shape_id", "route_short_name"]].drop_duplicates().head())


Summary of Current shapes_df Columns:
- shape_id: Unique shape identifier
- shape_pt_lat, shape_pt_lon: GPS points to trace the route shape
- shape_pt_sequence: Order of points to draw the shape
- shape_dist_traveled: Optional, often used to interpolate positions
- route_id, route_short_name: Now both included — route_short_name is the human-readable one you’ll want for plotting and filtering

In [ ]:
# --- Create base map ---
mymap = Map(location=[35.0844, -106.6504], zoom_start=12)

# --- Define color map for routes ---
route_names = sorted(df_anomalies_full['route_short_name'].dropna().unique())
cmap = plt.get_cmap("tab20", len(route_names))
color_map = {route: mcolors.to_hex(cmap(i)) for i, route in enumerate(route_names)}

# --- Plot anomaly markers by route and type ---
for (route, anomaly), subset in df_anomalies_full.groupby(["route_short_name", "anomaly_type"]):
    group = FeatureGroup(name=f"{route} – {anomaly}", show=False)
    for _, row in subset.iterrows():
        if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
            popup = (
                f"Anomaly: {anomaly}<br>"
                f"Route: {route}<br>"
                f"Vehicle: {row.get('vehicle_id', 'N/A')}<br>"
                f"Timestamp: {row.get('timestamp_collected', 'N/A')}"
            )
            CircleMarker(
                location=[row["latitude"], row["longitude"]],
                radius=4,
                color=color_map.get(route, "black"),
                fill=True,
                fill_opacity=0.9,
                popup=popup
            ).add_to(group)
    group.add_to(mymap)

# --- Plot jump lines and markers ---
if 'jumps_df' in globals() and not jumps_df.empty:
    for route, group_df in jumps_df.groupby("route_short_name"):
        jump_line_group = FeatureGroup(name=f"{route} – Jump Lines", show=False)
        jump_point_group = FeatureGroup(name=f"{route} – Jump Start/End", show=False)
        for _, row in group_df.iterrows():
            if all(pd.notna([row["lat_prev"], row["lon_prev"], row["latitude"], row["longitude"]])):
                start = [row["lat_prev"], row["lon_prev"]]
                end = [row["latitude"], row["longitude"]]
                vehicle = row.get("vehicle_id", "N/A")
                t_prev = row.get("timestamp_prev", "N/A")
                t_curr = row.get("timestamp_curr", "N/A")

                PolyLine([start, end], color="orange", weight=2,
                         tooltip=f"Vehicle {vehicle} jump").add_to(jump_line_group)

                CircleMarker(location=start, radius=4, color="blue", fill=True,
                             fill_opacity=0.9, popup=f"START – {vehicle}<br>{t_prev}").add_to(jump_point_group)
                CircleMarker(location=end, radius=4, color="purple", fill=True,
                             fill_opacity=0.9, popup=f"END – {vehicle}<br>{t_curr}").add_to(jump_point_group)
        jump_line_group.add_to(mymap)
        jump_point_group.add_to(mymap)

# --- Plot disappearances ---
if 'disappear_df' in globals() and not disappear_df.empty:
    for route, group_df in disappear_df.groupby("route_short_name"):
        disappear_group = FeatureGroup(name=f"{route} – Disappearances", show=False)
        for _, row in group_df.iterrows():
            if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
                Marker(
                    location=[row["latitude"], row["longitude"]],
                    icon=Icon(color="red", icon="times-circle", prefix="fa"),
                    tooltip=f"Vehicle {row.get('vehicle_id')} disappeared<br>{row.get('timestamp_collected')}"
                ).add_to(disappear_group)
        disappear_group.add_to(mymap)

# --- Plot reappearances ---
if 'reappear_df' in globals() and not reappear_df.empty:
    for route, group_df in reappear_df.groupby("route_short_name"):
        reappear_group = FeatureGroup(name=f"{route} – Reappearances", show=False)
        for _, row in group_df.iterrows():
            if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
                Marker(
                    location=[row["latitude"], row["longitude"]],
                    icon=Icon(color="green", icon="check-circle", prefix="fa"),
                    tooltip=f"Vehicle {row.get('vehicle_id')} reappeared<br>{row.get('timestamp_collected')}"
                ).add_to(reappear_group)
        reappear_group.add_to(mymap)

# --- Plot route shapes ---
if 'shapes_df' in globals() and not shapes_df.empty:
    for route, group in shapes_df.groupby("route_short_name"):
        route_group = FeatureGroup(name=f"{route} – Route Shape", show=False)
        for shape_id, shape_data in group.groupby("shape_id"):
            shape_data = shape_data.sort_values("shape_pt_sequence")
            latlons = list(zip(shape_data["shape_pt_lat"], shape_data["shape_pt_lon"]))
            PolyLine(
                locations=latlons,
                color=color_map.get(route, "gray"),
                weight=2,
                opacity=0.6,
                popup=f"Route {route} | Shape {shape_id}"
            ).add_to(route_group)
        route_group.add_to(mymap)

# --- Add layer controls ---
LayerControl(collapsed=False).add_to(mymap)

# --- Display the map ---
mymap